<div align=center dir=rtl>
    <h2>Scrapping world rank of Universities from Shanghi ranking website</h2>
    www.shanghairanking.com
</div>

by: Abdullah Mohammadi

abdhmohammady@gmail.com

https://github.com/AbdhMohammadi

<h3>Load laibraries and init data</h3>

In [ ]:

from urllib.request import urlopen
import re
import numpy as np
import pandas as pd

# Local file path to store data,
# This is my path in my laptop, change this path for your specified path
file_path = str('D:/DATA SCIENCE/05 - Text Mining/Web scrapping/Shanghi-University-Ranking/')
# url to scrapp
url_shanghairanking = ['http://www.shanghairanking.com/rankings/gras/2020/RS0210',
                       'http://www.shanghairanking.com/rankings/gras/2021/RS0210',
                       'http://www.shanghairanking.com/rankings/gras/2022/RS0210']


In the above, It is provided the  links of 3 pages of Shanghi ranking website for 2020, 2021, 2022<br>
to avoid of connection failures, we scrapp the pages first, next save the data in the local text files<br>
each file just contains <b>'tr'</b> tags of the main datatable in the source web pages.

<h3>Step 1: Scrapping the data</h3>

In [ ]:
# scrapping 3 urls from above links
for i in range(3):
  page = urlopen(url_shanghairanking[i])

  html_bytes = page.read()

  html_string = html_bytes.decode("utf-8")

  # find start and end of the main table tags 
  start_table = html_string.index('<tbody')
  end___table = html_string.index('</tbody>')
  # Get main body of the datatable
  tb = html_string[start_table:end___table]
  # remove tbody tag
  tb = re.sub('<tbody.*?>','',tb)
  # storing html string in the local file to avoid uknown connection failures
  f = open(file_path + str(2020 +i)+'.txt','w')
  f.write(tb)
  f.close()

<h3>Step 2 : 3 files must be read for years of 2020, 2021, 2022</h3>
We load content of each file in the array named 'source'.

In [ ]:
# list of data for each year 2020,2021,2022 
source = []
for i in range(3):

    f = open(file_path + str(2020+ i)+'.txt','r')
    
    source.append(str(f.buffer.read()))

    f.close()

<h3>Step 3: Processing the data</h3>
Below code contains 3 loop for processing html tags of the data.<br>
Loop 1: This loop selects a data associated to a file(2020, 2021, 2022)<br>
Loop 2: This loop chooses a tr tag from selected datatable<br>
Loop 3: Main task is proceeded here, each cell of the rows must be proceeded in this point,<br>
therfore we lookup the row in depth. the rows of the data contains 'Country/Region' cell that<br>
value is loaded as the image of the flag, we try first to scrapp country name from Flag image<br>
file name. next we proceed other cells value.

In [ ]:
data_ = []
columns_ =['Year','World Rank','Institution', 'Country/Region','Total Score', 'Q1']

# Reads 3 files
for i in range(3):

  # selects indexed file
  table = source[i]
  # proceed tag of <tr...>...</tr>
  while re.sub('<.*?>','',table)!='' :

    start_tr = table.index('<tr')
    end___tr = table.index('</tr>') + 5
    # select a row (tag of tr)
    tr_str = str(table[start_tr:end___tr])

    # Here tr_str contains tag of <tr..><td..>..</td><td..>..</td>..</tr>
    row = []
    row.append(2020 + i)

    # we need process 5 tag named as td, process ends when sub__ contains no value
    # <td data-v-ae1ab4a8><div class="ranking top1" data-v-ae1ab4a8>1</div></td>

    exit_condistion = False

    while not exit_condistion:

      start_td = tr_str.index('<td')
      end___td = tr_str.index('</td>') + 5

      # each td_str contains one tag of the <td..>..</td>
      td_str = str(tr_str[start_td:end___td])

      region_idx = td_str.find('region-img',0)
      
      # lookup for Country/Region tag
      if region_idx > -1:
         i1 = td_str.find('png100')
         i2 = td_str.find('png)')
         country_region = td_str[i1+7:i2-1]
         row.append(country_region.upper())
      else:

        # Removeing extra information:
        td_value = str(re.sub('<.*?tooltiptext.*?>','',td_str))
        # Removeing html charachters
        td_value = str(re.sub('<.*?>','',td_value))
        # removeing extra line break charachters
        td_value = td_value.replace('\\n','')
        td_value = td_value.replace('\\r','')
        # add retrived value as acell to the row
        row.append(td_value)

      # Normalize tr tag by close char
      tr_str = tr_str[end___td:-1] + '>'

      # apply exit loop conditions
      condistion = re.sub('<.*?>','',tr_str)
      # No blank space
      condistion = condistion.replace(' ','')
      # No line break \n
      condistion = condistion.replace('\\n','')
      # No line break \r
      condistion = condistion.replace('\\r','')
      # string must be empty
      exit_condistion = condistion == ''

    # adding the row to the collection
    data_.append(row)

    # Normalize reminded string of the table by close tag </tr>
    table = table[end___tr:-1]+'</tr>'

print('All data:', len(data_))
print('Ordered data using \'Total Score\' and \'Year\'')

All data: 90
Ordered data using 'Total Score' and 'Year'


<h3>Step 4: Display the Ranking data</h3>

In [ ]:
# Generate DataFrame
df = pd.DataFrame(columns = columns_, data=data_)
# Sort the dataframe by the score and year number
df.sort_values(by=['Total Score','Year'],ascending=[False,True],inplace=True)

# disply 30 row of dataframe
df.head(30)


,Year,World Rank,Institution,Country/Region,Total Score,Q1
0,2020,1,Massachusetts Institute of Technology (MIT),US,357.4,66.2
30,2021,1,Massachusetts Institute of Technology (MIT),US,355.3,61.1
60,2022,1,Massachusetts Institute of Technology (MIT),US,354.5,58.3
61,2022,2,Stanford University,US,320.6,53.8
31,2021,2,Stanford University,US,315.6,55.6
1,2020,2,Stanford University,US,308.1,62.4
2,2020,3,"University of California, Berkeley",US,291.3,51.8
62,2022,3,"University of California, Berkeley",US,290.8,44.7
32,2021,3,"University of California, Berkeley",US,284.4,46.9
3,2020,4,Carnegie Mellon University,US,283.6,60.2
